In [1]:
# reddit api
import praw
# data processing
import pandas as pd
import numpy as np
# helper modules
import time
# database access
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
cred = credentials.Certificate('./service_acct_key.json')
firebase_admin.initialize_app(cred, {
  'databaseURL': 'https://meme-room-default-rtdb.firebaseio.com/'
})

In [2]:
# initialize praw
reddit = praw.Reddit(
    client_id='hS4CPqNExizMF7XJ1XlMBQ',
    client_secret='zMcrLvT31UBI6V-zl2IFC9uR3MRA-g',
    user_agent='reddit-meme-analysis 0.1.1 by /u/inkoh',
    username='inkoh',
    password='Songoku777'
)

reddit.read_only = True


Version 7.3.0 of praw is outdated. Version 7.4.0 was released 2 days ago.


In [3]:
subreddit_index = pd.read_csv('subreddit-index-proto.csv')
subreddits = subreddit_index["real_name"]
bulk_posts = pd.read_csv('scraped_posts_bulk.csv')

In [7]:
def generate_subreddit_index(subreddits):
    i = 0
    l = len(subreddits)
    index = []
    for sub_name in subreddits:
        time.sleep(10)
        index.append(
            analyze_subreddit(sub_name)
        )
        print(sub_name + " analyzed successfully [" + str(i) + "/" + str(l) + "]")
        i += 1
    
    return index


def analyze_subreddit(sub_name):
    # request information about subreddit
    try:
        subdata = reddit.subreddit(sub_name)
        sampled_posts = bulk_posts[bulk_posts["subreddit"] == sub_name]
        sub_data = {
            "name": sub_name,
            "permalink": "https://reddit.com/r/" + sub_name,
            "subscribers": subdata.subscribers,
            "posts_analyzed": len(sampled_posts),
            "upvote_data": get_upvote_data(sampled_posts),
            "comment_data": get_comment_data(sampled_posts),
            "content_creator_diversity_ratio": get_ccdr(sampled_posts)
        }
        return sub_data
    except:
        print(sub_name + ": sub not found :(")
        return { "error_message": "subreddit not found" }


def get_upvote_data(post_data):
    upvote_scores = np.array(post_data["num_upvotes"])
    return {
        "avg_upvotes": float(upvote_scores.mean()),
        "min_upvotes": float(upvote_scores.min()),
        "max_upvotes": float(upvote_scores.max()),
        "std": float(upvote_scores.std())
    }


def get_comment_data(post_data):
    comment_scores = np.array(post_data["num_comments"])
    return {
        "avg_comments": float(comment_scores.mean()),
        "std": float(comment_scores.std()),
        "min_comments": float(comment_scores.min()),
        "max_comments": float(comment_scores.max()),
    }


def get_ccdr(post_data):
    authors = post_data["author"]
    num_posts = len(post_data)
    num_unique_authors = len(set(authors))
    return num_unique_authors / num_posts


In [ ]:
processed_index = generate_subreddit_index(subreddits)

In [11]:
for sub_data in processed_index:
  try:
    sub_name = sub_data['name']
    db.reference('/subreddit_index_2/' + sub_name).set(sub_data)
  except:
    print('skipping this broken datapoint')

skipping this broken datapoint
skipping this broken datapoint
skipping this broken datapoint
skipping this broken datapoint
skipping this broken datapoint
skipping this broken datapoint
skipping this broken datapoint


In [12]:
d = pd.read_json('./subreddit-index-final.json')
d.pop('error_message')
d = d.to_dict(orient="records")

for sub_data in d:
  db.reference('/subreddit_index/' + sub_data['name']).set(sub_data)

# db.reference('/subreddit_index').set(dict_index)

